In [1]:
import csv

f = open('result.csv', 'r', encoding='UTF8')
rdr = csv.reader(f)

song_lyrics = {}
for line in rdr:
    song_lyrics[line[0]] = line[1]
f.close()
song_lyrics.pop('0')
song_lyrics

{'INVU': 'Falling in love너에겐 난 Option시작부터 다른 너와 나깨지는 Heart빗나간 네 Mention익숙하거든I think I lost my mindBut It’s my kind of love아끼지 않고 다쏟아내고는 주저앉아문득 어느 순간지친 내가 보여애쓰고 있지만So I can’t love youEven though I do밀어내 봐도난 너를 못 이겨날 버리고날 잃을수록넌 반짝이는 아이러니So I N V UI N V UI N V U기대지 마기대하지도 마몇 번을 되뇌고 되놰도그 손길 한 번에무너지는 날 볼 때네 기분은 어때I guess I lost my mindYeah It’s my kind of love늘 처음인 것처럼서툴러서 또 상처가 나무뎌지기 전에아물기도 전에잔뜩 헤집어놔So I can’t love youEven though I do밀어내 봐도난 너를 못 이겨날 버리고날 잃을수록넌 반짝이는 아이러니So I N V UI N V UI N V U식은 온기부서진 맘이자꾸 날 할퀴어아파도 못 멈춰So when you leavePlease make it easyCause I N V UI N V UI N V UI N V UI N V U',
 'RUN2U': 'RUN!Told you not 또 괜한 기대겉으론 걱정해 난 안 바뀌네또 가끔 말을 막 해 너무 딱해헛소리들 나는 안 들리네no oh yeahTold you 난 so always b day겉으론 내 편인데 못해 이해그 참견들은 가짜 나는 바빠어떻게 해도 나는 안 들리네no no oh타 버리고 파 너의 사랑은so sunny yeah사라져도 사라져도다 버리고 파 너만 있다면no worry yeah알잖아 It should be youSo I’LL RUN TO YOUSo I’LL RUN TO YOU선을 넘는 거래도 over and over다쳐도 괜찮아 I’LL RUN TO YOUSo I’LL RUN TO YOUA little bit little bit YoungA little bit litt

In [2]:
from konlpy.tag import Kkma

def lyrics_tokenizer(lyrics):
    kkma = Kkma()
    tokens = kkma.morphs(lyrics)
    token_refined = []
    stop_word = ['SF', 'SP', 'SS', 'SE', 'SO', 'SW', 'UN', 'OH', 'ON']
    for token in tokens:
        for w, t in kkma.pos(token):
            if t not in stop_word:
                token_refined.append(w)
    return token_refined

In [3]:
thema = ['비', '이별', '여행', '사랑', '눈', '외로움']

In [4]:
from gensim.models import FastText
from tqdm import tqdm

song_info = {}
for song in tqdm(song_lyrics):
    lyrics = song_lyrics[song]
    refined = lyrics_tokenizer(lyrics)
    model = FastText(sentences=[refined], window=5, min_count=1, sg=1, workers=4)
    thema_score = {}
    for t in thema:
        m_s = model.wv.most_similar(t)
        total_score = 0
        for w, s in m_s:
            total_score = total_score + s
        thema_score[t] = total_score
    song_thema= max(thema_score.keys(), key=(lambda k : thema_score[k]))
    song_info[song] = song_thema
song_info

100%|████████████████████████████████████████████████████████████████████████████| 6492/6492 [2:02:00<00:00,  1.13s/it]


{'INVU': '비',
 'RUN2U': '사랑',
 'ELEVEN': '눈',
 'Step Back': '사랑',
 'SMILEY (Feat. BIBI)': '외로움',
 'Next Level': '사랑',
 'Dreams Come True': '사랑',
 'Weekend': '외로움',
 'Savage': '외로움',
 'Butter': '눈',
 'Dynamite': '비',
 '라일락': '이별',
 'Permission to Dance': '사랑',
 'Celebrity': '사랑',
 "롤린 (Rollin')": '사랑',
 'Dun Dun Dance': '사랑',
 '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)': '눈',
 'Dilemma': '사랑',
 'DM': '이별',
 'ASAP': '사랑',
 'Queendom': '눈',
 'DUMB DUMB': '사랑',
 'O.O': '눈',
 'Lovesick Girls': '사랑',
 'WA DA DA': '사랑',
 'BOP BOP!': '사랑',
 '어푸 (Ah puh)': '비',
 '어른아이 (Toddler)': '이별',
 '색안경 (STEREOTYPE)': '눈',
 'LOVE DAY (2021) (바른연애 길잡이 X 양요섭, 정은지)': '사랑',
 '운전만해 (We Ride)': '이별',
 'Dolphin': '사랑',
 'How You Like That': '눈',
 'Black Mamba': '이별',
 '상상더하기': '사랑',
 'SCIENTIST': '사랑',
 'LOCO': '눈',
 '살짝 설렜어 (Nonstop)': '외로움',
 'Timeless': '비',
 'Alcohol-Free': '사랑',
 '치맛바람 (Chi Mat Ba Ram)': '눈',
 '마지막처럼': '사랑',
 '하루 끝': '사랑',
 'Rock with you': '이별',
 'Psycho': '사랑',
 'Forever Young': '눈',
 'You

In [5]:
import csv
f = open('song_info.csv', 'w', encoding ='UTF-8')
w = csv.writer(f)
for s in song_info:
    w.writerow([s, song_info[s]])
f.close()

In [6]:
import cx_Oracle

db = cx_Oracle.connect("ateam", "1234", "125.133.93.99:1521/xe")
sql = "update song set song_thema = 'sdf' where song_title = 'INVU'"
cursor = db.cursor()
cursor.execute(sql)
cursor.close()
db.commit()
db.close()

ModuleNotFoundError: No module named 'cx_Oracle'